In [1]:
import pandas as pd
import seaborn as sns
import re

In [2]:
prop_list = pd.read_csv("../data/raw/listing_properties.csv")

In [3]:
prop_list

,Unnamed: 0,beds,baths,parking,title,price,address,promoType,listingType,propertyType,status,mode,isRural,houses,bond,id,availableFrom,landArea,buildingArea,method
0,0,1,1,0,$330,$330,"409/757 Bourke, Docklands VIC 3008",basic,rent,Apartment / Unit / Flat,recentlyUpdated,rent,False,0,1433.0,14303265,NaN,NaN,NaN,NaN
1,1,2,1,1,$430 pw,$430 pw,"38/60-68 Gladesville Boulevard, Patterson Lake...",branded,rent,Apartment / Unit / Flat,new,rent,False,0,1868.0,16089992,"Monday, 28th June 2021",NaN,NaN,NaN
2,2,4,3,2,$585,$585,"1 Liverpool Circuit, Craigieburn VIC 3064",branded,rent,House,recentlyUpdated,rent,False,0,NaN,15279569,"Friday, 25th November 2022",NaN,NaN,NaN
3,3,3,1,1,$420,$420,"8 Stoneman Turn, Caroline Springs VIC 3023",branded,rent,House,new,rent,False,0,1820.0,16089977,"Monday, 26th September 2022",NaN,NaN,NaN
4,4,1,1,1,$640 per week,$640 per week,"1510/1 Freshwater Place, Southbank VIC 3006",platinum,rent,Apartment / Unit / Flat,new,rent,False,0,2781.0,16089972,"Monday, 22nd August 2022",NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13748,13748,4,2,2,$400 Per Week pw,$400 Per Week pw,"7 Ebony Way, Tarneit VIC 3029",basic,rent,House,live,rent,False,0,1738.0,15434092,"Wednesday, 27th October 2021",NaN,NaN,NaN
13749,13749,1,1,0,$435 per week,$435 per week,"17/416 St Kilda Rd, Melbourne VIC 3004",basic,rent,Apartment / Unit / Flat,live,rent,False,0,2500.0,14409092,"Saturday, 9th October 2021",NaN,50.0,NaN
13750,13750,3,2,4,$350 Per Week pw,$350 Per Week pw,"16 Goulburn Court, Werribee VIC 3030",basic,rent,House,live,rent,False,0,1521.0,15430756,"Wednesday, 6th October 2021",NaN,NaN,NaN
13751,13751,4,2,1,$600,$600,"18 Jessie Street, Preston VIC 3072",basic,rent,House,live,rent,False,0,2600.0,15428527,"Tuesday, 5th October 2021",NaN,NaN,NaN


In [4]:
prop_list[prop_list['landArea'].notna()]

,Unnamed: 0,beds,baths,parking,title,price,address,promoType,listingType,propertyType,status,mode,isRural,houses,bond,id,availableFrom,landArea,buildingArea,method
11,11,4,2,2,$630 weekly,$630 weekly,"90 Shepherd Rd, Glen Waverley VIC 3150",basic,rent,House,new,rent,False,0,2738.0,16089938,"Friday, 23rd September 2022",652.0,175.0,NaN
13,13,2,1,0,$700 per week,$700 per week,"15 Dally Street, Clifton Hill VIC 3068",basic,rent,House,recentlyUpdated,rent,False,0,3042.0,15478015,"Friday, 14th October 2022",158.0,95.0,NaN
14,14,4,2,2,$500,$500,"20 Knightsford Avenue, Clyde VIC 3978",branded,rent,House,new,rent,False,0,2173.0,16089930,"Wednesday, 12th October 2022",375.0,NaN,NaN
18,18,3,1,2,$360 Per Week,$360 Per Week,"5 Roxburgh Road, Yallourn North VIC 3825",branded,rent,House,new,rent,False,0,1560.0,16089911,"Wednesday, 5th October 2022",634.0,NaN,NaN
35,35,3,2,2,$460 pw,$460 pw,"2/17 Nelson Road, Trafalgar VIC 3824",basic,rent,Townhouse,new,rent,False,0,1999.0,16089810,"Sunday, 25th September 2022",317.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13667,13667,3,2,11,$1180 PER WEEK $5113 PER MONTH,$1180 PER WEEK $5113 PER MONTH,"2531 STEIGLITZ ROAD, Steiglitz VIC 3331",branded,rent,Acreage / Semi-Rural,live,rent,True,0,5000.0,15714546,"Sunday, 20th February 2022",600000.0,NaN,NaN
13670,13670,4,2,2,Contact Agent,Contact Agent,"13 Roaming Drive, Fraser Rise VIC 3336",branded,rent,House,live,rent,False,0,NaN,15711847,"Wednesday, 6th April 2022",352.0,NaN,NaN
13677,13677,2,2,1,UNDER APPLICATION,UNDER APPLICATION,"2/50 Fraser Crescent, Ocean Grove VIC 3226",branded,rent,Townhouse,live,rent,False,0,2086.0,15649364,"Saturday, 5th February 2022",197.0,NaN,NaN
13732,13732,4,2,2,$410.00 Per Week pw,$410.00 Per Week pw,"10 Clearwood Drive, Truganina VIC 3029",basic,rent,House,live,rent,False,0,1782.0,15454576,"Friday, 10th December 2021",360.0,22.0,NaN


In [4]:
prop_subset = prop_list[['id','price','beds','baths','parking','title','address','propertyType','status','bond']]

In [5]:
prop_subset.isnull().sum()

id                0
price             0
beds              0
baths             0
parking           0
title             0
address           0
propertyType      0
status            0
bond            890
dtype: int64

In [6]:
prop_subset_clean = prop_subset.dropna()

In [7]:
prop_subset_clean.isnull().sum()

id              0
price           0
beds            0
baths           0
parking         0
title           0
address         0
propertyType    0
status          0
bond            0
dtype: int64

In [8]:
prop_subset_clean

,id,price,beds,baths,parking,title,address,propertyType,status,bond
0,14303265,$330,1,1,0,$330,"409/757 Bourke, Docklands VIC 3008",Apartment / Unit / Flat,recentlyUpdated,1433.0
1,16089992,$430 pw,2,1,1,$430 pw,"38/60-68 Gladesville Boulevard, Patterson Lake...",Apartment / Unit / Flat,new,1868.0
3,16089977,$420,3,1,1,$420,"8 Stoneman Turn, Caroline Springs VIC 3023",House,new,1820.0
4,16089972,$640 per week,1,1,1,$640 per week,"1510/1 Freshwater Place, Southbank VIC 3006",Apartment / Unit / Flat,new,2781.0
6,16089969,$550 PER WEEK,4,2,2,$550 PER WEEK,"6 Plenty Drive, Warragul VIC 3820",House,new,2390.0
...,...,...,...,...,...,...,...,...,...,...
13748,15434092,$400 Per Week pw,4,2,2,$400 Per Week pw,"7 Ebony Way, Tarneit VIC 3029",House,live,1738.0
13749,14409092,$435 per week,1,1,0,$435 per week,"17/416 St Kilda Rd, Melbourne VIC 3004",Apartment / Unit / Flat,live,2500.0
13750,15430756,$350 Per Week pw,3,2,4,$350 Per Week pw,"16 Goulburn Court, Werribee VIC 3030",House,live,1521.0
13751,15428527,$600,4,2,1,$600,"18 Jessie Street, Preston VIC 3072",House,live,2600.0


In [10]:
def get_price(e):
    s = re.search('\$[0-9]*\,?\d*\.?\d*', e)
    if (s == None):
        return None
    else:
        final = s.group().strip('$')
        final = final.replace(',','')
        return final

In [11]:
prop_subset_clean['price'] = prop_subset_clean['price'].apply(get_price)

/tmp/ipykernel_392/2730741709.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prop_subset_clean['price'] = prop_subset_clean['price'].apply(get_price)


In [12]:
prop_subset_clean

,id,price,beds,baths,parking,title,address,propertyType,status,bond
0,14303265,330,1,1,0,$330,"409/757 Bourke, Docklands VIC 3008",Apartment / Unit / Flat,recentlyUpdated,1433.0
1,16089992,430,2,1,1,$430 pw,"38/60-68 Gladesville Boulevard, Patterson Lake...",Apartment / Unit / Flat,new,1868.0
3,16089977,420,3,1,1,$420,"8 Stoneman Turn, Caroline Springs VIC 3023",House,new,1820.0
4,16089972,640,1,1,1,$640 per week,"1510/1 Freshwater Place, Southbank VIC 3006",Apartment / Unit / Flat,new,2781.0
6,16089969,550,4,2,2,$550 PER WEEK,"6 Plenty Drive, Warragul VIC 3820",House,new,2390.0
...,...,...,...,...,...,...,...,...,...,...
13748,15434092,400,4,2,2,$400 Per Week pw,"7 Ebony Way, Tarneit VIC 3029",House,live,1738.0
13749,14409092,435,1,1,0,$435 per week,"17/416 St Kilda Rd, Melbourne VIC 3004",Apartment / Unit / Flat,live,2500.0
13750,15430756,350,3,2,4,$350 Per Week pw,"16 Goulburn Court, Werribee VIC 3030",House,live,1521.0
13751,15428527,600,4,2,1,$600,"18 Jessie Street, Preston VIC 3072",House,live,2600.0


In [13]:
prop_subset_clean['price'] = pd.to_numeric(prop_subset_clean['price'])

/tmp/ipykernel_392/2753503525.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prop_subset_clean['price'] = pd.to_numeric(prop_subset_clean['price'])


In [14]:
prop_subset_clean

,id,price,beds,baths,parking,title,address,propertyType,status,bond
0,14303265,330.0,1,1,0,$330,"409/757 Bourke, Docklands VIC 3008",Apartment / Unit / Flat,recentlyUpdated,1433.0
1,16089992,430.0,2,1,1,$430 pw,"38/60-68 Gladesville Boulevard, Patterson Lake...",Apartment / Unit / Flat,new,1868.0
3,16089977,420.0,3,1,1,$420,"8 Stoneman Turn, Caroline Springs VIC 3023",House,new,1820.0
4,16089972,640.0,1,1,1,$640 per week,"1510/1 Freshwater Place, Southbank VIC 3006",Apartment / Unit / Flat,new,2781.0
6,16089969,550.0,4,2,2,$550 PER WEEK,"6 Plenty Drive, Warragul VIC 3820",House,new,2390.0
...,...,...,...,...,...,...,...,...,...,...
13748,15434092,400.0,4,2,2,$400 Per Week pw,"7 Ebony Way, Tarneit VIC 3029",House,live,1738.0
13749,14409092,435.0,1,1,0,$435 per week,"17/416 St Kilda Rd, Melbourne VIC 3004",Apartment / Unit / Flat,live,2500.0
13750,15430756,350.0,3,2,4,$350 Per Week pw,"16 Goulburn Court, Werribee VIC 3030",House,live,1521.0
13751,15428527,600.0,4,2,1,$600,"18 Jessie Street, Preston VIC 3072",House,live,2600.0


In [15]:
prop_subset_clean = prop_subset_clean.dropna(axis=0)

In [21]:
prop_subset_clean = prop_subset_clean.reset_index()

In [22]:
prop_subset_clean['price'].sort_values(ascending = True)

7862        40.0
10950       40.0
12075       50.0
4277        50.0
12350       50.0
          ...   
6364      4000.0
11403     5000.0
7512      5500.0
10550    39000.0
11644    41250.0
Name: price, Length: 12487, dtype: float64

In [23]:
prop_subset_clean.iloc[11644,:]

index                                                       12751
id                                                       15994278
price                                                     41250.0
beds                                                            0
baths                                                           0
parking                                                         0
title                                                   $41250 pa
address         Level 1, 301/13-15 Lake Street, Caroline Sprin...
propertyType                              Apartment / Unit / Flat
status                                                       live
bond                                                       9519.0
Name: 11644, dtype: object

In [26]:
prop_subset_clean = prop_subset_clean[prop_subset_clean['beds'] > 0]

In [29]:
prop_subset_clean.pop('index')

0            0
1            1
2            3
3            4
4            6
         ...  
12482    13748
12483    13749
12484    13750
12485    13751
12486    13752
Name: index, Length: 12405, dtype: int64

In [32]:
prop_subset_clean.to_csv('../data/curated/cleaned_property_listing_data.csv')